# Creación de base de datos para identificación de HVs

El presente archivo se creará la base de datos extrayendo información de HVs usando procesamiento de lenguaje natural. Esto con el objetivo de más adelante tener la posibilidad de generar con base en 300 HVs tanto rechazadas como avanzadas, un modelo que nos permita identificar las características predominantes de una HV que hace que avance o no durante el proceso.



#### Importar librerias

In [20]:
import os
import fitz  # PyMuPDF for PDFs
import pytesseract
from PIL import Image
import pandas as pd
import re
import spacy

### Importar SpaCy para NLP
Spacy es una librería de python que permite por medio de modelos de lenguaje pre-importados realizar análisis de texto, identificando palabras, nombres, lugares, objetos, verbos, adjetivos y la relación entre los mismos.

En este caso, importamos el modelo pre-entrenado en inglés, lo que requiere que todas las CVs a procesar estén en este idioma.


In [21]:
nlp = spacy.load("en_core_web_sm")

#### Cargar las carpetas con las HVs

In [22]:
hv_dir_exitosas_java = "hojas_de_vida/java/Paso"
hv_dir_noexitosas_java = "hojas_de_vida/java/No Paso"
hv_dir_exitosas_front = "hojas_de_vida/frontend/Paso"
hv_dir_noexitosas_front = "hojas_de_vida/frontend/No Paso"

### Deifinir palabras clave
En este caso, se definirar palabras clave que podrán tener las HVs teniendo en cuenta que para este modelo en particular se está utlizando solo HVs para un requerimiento de **desarrolladores Java**.


In [23]:
palabras_clave_java = ["Java", "Spring", "spring boot", "AWS", "Azure", "GCP", "Google Cloud Platform", "microservices", "Maven", "Gradle", "Java Server Pages", "JSP", "JEE", "Java Enterprise Edition", "Java8", "Java11", "Java17", "Java21", "JVM", "Java virtual machine"]

palabras_clave_front_end = ["Javascript", "Typescript", "React", "Angular", "Vue", "react.js", "vue.js", "HTML", "CSS", "Redux", "Hooks", "Micro frontends"]

### Definir las secciones y los patrones en las que estas van a aparecer
Además de definir la cantidad de palabras clave, es importante contar con las secciones con las que cada documento puede contar y entender si cuenta o no con este.

In [24]:
secciones = {
    "education": r"education|academic background|studies|study|university studies|professional education",
    "work_experience": r"work experience|employment history|professional experience|background|professional background",
    "skills": r"skills|technical skills|competencies",
    "certifications": r"certifications|licenses|accreditations",
    "achievements": r"achievements|achieved",
    "professional_profile": r"profile|summary|about me|professional summary|objective|summary",
    "languages": r"languages|linguistic skills|spoken languages",
    "projects": r"projects|case studies|portfolio",
    "publications": r"publications|research papers|articles|books",
    "training_courses": r"training|courses|workshops|online learning",
    "volunteer_work": r"volunteer|volunteering|social impact|community service",
}

### Detectar el tipo de HV para posterior procesamiento de palabras clave


In [25]:
def detect_cv_type(cv_path):
    if "java" in cv_path.lower():
        return "java"
    elif "frontend" in cv_path.lower():
        return "frontend"
    else:
        return "unknown"  # Default case if it's unclear

### Extraer el texto de los PDFs

A continuación se usará la librería FITZ, la cual ayuda a extraer el texto de un PDF, ver si tiene imágenes, contar sus páginas y detectar colores en los mismos.

La declaramos como función para llamarla más adelante en el procesamiento de todas las características que buscamos extraer.

In [26]:
def extraer_texto_pdf(pdf_path):
    text = ""
    try:
        doc = fitz.open(pdf_path)
        for page in doc:
            text += page.get_text("text") + "\n"
    except Exception as e:
        print(f"Leyendo PDF {pdf_path}: {e}")
    return text.strip()

### Contar palabras en general

In [27]:
def contar_palabras(text):
    return len(text.split()) if text else 0

#### Contar palabras clave

In [28]:
def contar_palabras_clave(text, cv_type):
    text_lower = text.lower()

    # Use the appropriate keyword list
    if cv_type == "java":
        keyword_list = palabras_clave_java
    elif cv_type == "frontend":
        keyword_list = palabras_clave_front_end
    else:
        keyword_list = []  # Default case (shouldn't happen)

    return sum(1 for keyword in keyword_list if keyword.lower() in text_lower)

#### Extraer las secciones

Para extraer las secciones, usamos expresiones regulares. Con la biblioteca Re, busca el patron definido en la variable secciones más arriba, que ayuda a identificar si el texto obtenido del PDF tiene o no esta sección.

In [29]:
import re

def extraer_secciones(text):
    sections = {key: {"exists": False, "word_count": 0} for key in secciones.keys()}

    for section, pattern in secciones.items():
        match = re.search(pattern, text, re.IGNORECASE)
        if match:
            sections[section]["exists"] = True  # Section exists
            section_start = match.start()
            next_match = min(
                (m.start() for s, p in secciones.items() if (m := re.search(p, text[section_start + 1:], re.IGNORECASE))),
                default=len(text)
            )
            sections[section]["word_count"] = contar_palabras(text[section_start:section_start + next_match])

    return sections

### Verificar factores como foto y colores
De vuelta se usa la librería fitz para poder leer el PDF

#### Verificar si tiene o no foto

In [30]:
def tiene_foto_pdf(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        for page in doc:
            if len(page.get_images(full=True)) > 0:
                return True
    except Exception as e:
        print(f"Error revisando foto en PDF {pdf_path}: {e}")
    return False

#### Verificar si tiene colores adicionales el PDF

In [31]:
def tiene_color_pdf(pdf_path):
    doc = fitz.open(pdf_path)

    for page in doc:
        for draw in page.get_drawings():
            if "color" in draw:
                return True

    return False

#### Contar páginas

In [32]:
def contar_paginas(pdf_path):
    try:
        doc = fitz.open(pdf_path)
        return len(doc)
    except Exception as e:
        print(f"Error counting pages in PDF {pdf_path}: {e}")
        return 1

### Procesamiento del CV
A continuación la función de procesamiento, nos ayudará a procesar un solo CV de acuerdo a los parámetros establecidos anteriormente, ejecutando cada una de las funciones ya establecidas

In [33]:
def process_cv(cv_path):
    text = ""
    has_photo = False
    has_colors = False
    num_pages = 1

    tipo_cv = detect_cv_type(cv_path)

    text = extraer_texto_pdf(cv_path)
    has_photo = tiene_foto_pdf(cv_path)
    has_colors = tiene_color_pdf(cv_path)
    num_pages = contar_paginas(cv_path)

    if not text:
        print(f"No se pudo extraer texto de {cv_path}")

    # Extract features
    total_word_count = contar_palabras(text)
    keyword_count = contar_palabras_clave(text, tipo_cv)
    sections = extraer_secciones(text)

    return {
        "CV_Name": os.path.basename(cv_path),
        "Total_Word_Count": total_word_count,
        "Has_Photo": int(has_photo),
        "Has_Colors": int(has_colors),
        "Pages": num_pages,
        "Keyword_Count": keyword_count,
        "Education_Exists": int(sections["education"]["exists"]),
        "Education_Word_Count": sections["education"]["word_count"],
        "Work_Experience_Exists": int(sections["work_experience"]["exists"]),
        "Work_Experience_Word_Count": sections["work_experience"]["word_count"],
        "Skills_Exists": int(sections["skills"]["exists"]),
        "Skills_Word_Count": sections["skills"]["word_count"],
        "Certifications_Exists": int(sections["certifications"]["exists"]),
        "Certifications_Word_Count": sections["certifications"]["word_count"],
        "Achievements_Exists": int(sections["achievements"]["exists"]),
        "Achievements_Word_Count": sections["achievements"]["word_count"],
        "Professional_Profile_Exists": int(sections["professional_profile"]["exists"]),
        "Professional_Profile_Word_Count": sections["professional_profile"]["word_count"],
        "Projects_Exists": int(sections["projects"]["exists"]),
        "projects_Word_Count": sections["projects"]["word_count"],
        "volunteer_work_Exists": int(sections["volunteer_work"]["exists"]),
        "volunteer_work_Word_Count": sections["volunteer_work"]["word_count"]
    }

### Procesamiento de CVs en la carpeta
La siguiente función nos ayuda a de acuerdo con lo establecido anteriormente, procesar todas las CVs en las carpetas seleccionadas y devolverlas en una lista

In [34]:
def process_folder(folder_path, label):
    cv_data = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".pdf"):
            cv_path = os.path.join(folder_path, filename)
            print(f"Processing: {cv_path}")
            cv_info = process_cv(cv_path)
            cv_info["Passed"] = label
            cv_data.append(cv_info)
    return cv_data

## Creación de la base de datos

Se crean las variables donde se almacenan las CVs exitosas procesadas, agregando la información de 1 si es exitosa y 0 si no es exitosa.

In [35]:
successful_data_java = process_folder(hv_dir_exitosas_java, 1)  # Label = 1 (Passed)
unsuccessful_data_java = process_folder(hv_dir_noexitosas_java, 0)  # Label = 0 (Not Passed)

successful_data_front = process_folder(hv_dir_exitosas_front, 1)  # Label = 1 (Passed)
unsuccessful_data_front = process_folder(hv_dir_noexitosas_front, 0)  # Label = 0 (Not Passed)

Processing: hojas_de_vida/java/Paso\11686212-CV-Jorge Vidal.pdf
Processing: hojas_de_vida/java/Paso\14483303-Andres_Gomez_resume.docx (2) (1).pdf
Processing: hojas_de_vida/java/Paso\14503315-CV - Henry Luis Gomez Ortiz [En] (1) (1).pdf
Processing: hojas_de_vida/java/Paso\14955159-Curriculum vitae english (1).pdf
Processing: hojas_de_vida/java/Paso\8A9EBB5F-08ED-4D90-92C2-72A9024C2A58.pdf
Processing: hojas_de_vida/java/Paso\985269fa-a528-46aa-bc9c-403ee23fbcd3_CV Felipe Feres .pdf
Processing: hojas_de_vida/java/Paso\ABD57CA0-6506-4FF2-84B3-42F845509854.pdf
Processing: hojas_de_vida/java/Paso\Agustin Castro CV.pdf
Processing: hojas_de_vida/java/Paso\Alex Diaz CV - 2022.pdf
Processing: hojas_de_vida/java/Paso\alexis_pequeno_cv_en.pdf
Processing: hojas_de_vida/java/Paso\BBD1018F-127C-4C11-98CA-12E050D26B3B.pdf
Processing: hojas_de_vida/java/Paso\Byron Andrago CV.pdf
Processing: hojas_de_vida/java/Paso\Carlos Pinto Jimenez_CV.docx.pdf
Processing: hojas_de_vida/java/Paso\Christian_Blanco_CV_

Se guarda esta información en un dataframe

In [40]:
data_total = successful_data_java + unsuccessful_data_java + successful_data_front + unsuccessful_data_front
baseCVs = pd.DataFrame(data_total)

#borramos el CV name ya que no es necesaria y buscamos información anónima
baseCVs = baseCVs.drop('CV_Name', axis=1)

#Aleatorizamos el orden para que no queden juntos los 1 y los 0 todos juntos y las muestras sean más representativas
baseCVs = baseCVs.sample(frac=1, random_state=42).reset_index(drop=True)


In [41]:
baseCVs

,Total_Word_Count,Has_Photo,Has_Colors,Pages,Keyword_Count,Education_Exists,Education_Word_Count,Work_Experience_Exists,Work_Experience_Word_Count,Skills_Exists,...,Certifications_Word_Count,Achievements_Exists,Achievements_Word_Count,Professional_Profile_Exists,Professional_Profile_Word_Count,Projects_Exists,projects_Word_Count,volunteer_work_Exists,volunteer_work_Word_Count,Passed
0,566,0,1,3,9,1,49,0,0,1,...,2,0,0,0,0,0,0,0,0,1
1,308,0,1,1,7,1,14,1,86,0,...,0,0,0,0,0,0,0,0,0,1
2,378,0,1,1,7,1,1,1,1,1,...,0,0,0,1,24,0,0,0,0,0
3,330,0,1,1,6,1,131,1,104,1,...,0,0,0,1,17,1,14,0,0,0
4,277,0,1,2,3,1,22,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
504,2185,1,1,7,9,1,15,1,106,1,...,0,0,0,1,103,1,8,0,0,1
505,573,0,1,2,9,1,29,0,0,0,...,0,0,0,1,51,0,0,0,0,0
506,561,1,1,3,4,1,65,1,58,1,...,0,0,0,1,10,1,113,0,0,1
507,607,1,1,1,10,1,118,0,0,1,...,0,0,0,1,3,1,52,0,0,0


### Exportar base en un archivo CSV para posterior lectura

In [42]:
baseCVs.to_csv("baseCVs.csv", index=False)